In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load the customer dataset
df = pd.read_csv('data/online_retail.csv', encoding='latin1')

# Basic dataset information
print("Dataset Shape:", df.shape)
print("\nColumn Names and Data Types:")
print(df.dtypes)
print("\nFirst 5 rows:")
print(df.head())
print("\nDataset Info:")
df.info()
print("\nMissing Values:")
print(df.isnull().sum())

# Unique values analysis for customer segmentation
print(f"\nUnique Customers: {df['CustomerID'].nunique()}")
print(f"Unique Products: {df['StockCode'].nunique()}")
print(f"Unique Countries: {df['Country'].nunique()}")
print(f"Date Range: {df['InvoiceDate'].min()} to {df['InvoiceDate'].max()}")

# Basic statistics
print("\nNumerical Statistics:")
print(df.describe())

# Check for negative quantities (returns)
print(f"\nNegative Quantities (Returns): {len(df[df['Quantity'] < 0])}")
print(f"Zero Unit Price: {len(df[df['UnitPrice'] <= 0])}")

Dataset Shape: (541909, 8)

Column Names and Data Types:
InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

First 5 rows:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4 

In [4]:
# Customer behavior overview (for non-null CustomerID)
customer_data = df[df['CustomerID'].notna()].copy()

print(f"Records with CustomerID: {len(customer_data)}")
print(f"Total Revenue: ${(customer_data['Quantity'] * customer_data['UnitPrice']).sum():,.2f}")

# Preview customer transaction patterns
customer_preview = customer_data.groupby('CustomerID').agg({
    'InvoiceDate': ['min', 'max', 'count'],
    'Quantity': 'sum',
    'UnitPrice': 'mean'
}).head(10)

print("\nSample Customer Transaction Patterns:")
print(customer_preview)

Records with CustomerID: 406829
Total Revenue: $8,300,065.81

Sample Customer Transaction Patterns:
                InvoiceDate                        Quantity  UnitPrice
                        min              max count      sum       mean
CustomerID                                                            
12346.0     1/18/2011 10:01  1/18/2011 10:17     2        0   1.040000
12347.0     1/26/2011 14:30    8/2/2011 8:48   182     2458   2.644011
12348.0     1/25/2011 10:42  9/25/2011 13:13    31     2341   5.764839
12349.0     11/21/2011 9:51  11/21/2011 9:51    73      631   8.289041
12350.0      2/2/2011 16:01   2/2/2011 16:01    17      197   3.841176
12352.0     11/3/2011 14:37  9/28/2011 14:58    95      470  23.274737
12353.0     5/19/2011 17:47  5/19/2011 17:47     4       20   6.075000
12354.0     4/21/2011 13:11  4/21/2011 13:11    58      530   4.503793
12355.0      5/9/2011 13:49   5/9/2011 13:49    13      240   4.203846
12356.0      1/18/2011 9:50   4/8/2011 12:33    

In [5]:
# Start with customer data (non-null CustomerID only)
df_clean = customer_data.copy()

# Convert InvoiceDate to datetime
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

# Remove returns (negative quantities) and zero prices for positive sales analysis
df_clean = df_clean[(df_clean['Quantity'] > 0) & (df_clean['UnitPrice'] > 0)].copy()

# Calculate total amount per transaction
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['UnitPrice']

# Remove outliers using IQR method for Quantity and UnitPrice
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply outlier removal
df_clean = remove_outliers(df_clean, 'Quantity')
df_clean = remove_outliers(df_clean, 'UnitPrice')
df_clean = remove_outliers(df_clean, 'TotalAmount')

print("Data Cleaning Summary:")
print(f"Original records with CustomerID: {len(customer_data):,}")
print(f"After removing returns and zero prices: {len(df_clean):,}")
print(f"Records removed: {len(customer_data) - len(df_clean):,}")
print(f"Final dataset shape: {df_clean.shape}")

# Verify data quality
print(f"\nDate range: {df_clean['InvoiceDate'].min()} to {df_clean['InvoiceDate'].max()}")
print(f"Unique customers: {df_clean['CustomerID'].nunique():,}")
print(f"Total revenue: ${df_clean['TotalAmount'].sum():,.2f}")
print(f"Average order value: ${df_clean['TotalAmount'].mean():.2f}")

Data Cleaning Summary:
Original records with CustomerID: 406,829
After removing returns and zero prices: 324,716
Records removed: 82,113
Final dataset shape: (324716, 9)

Date range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00
Unique customers: 4,146
Total revenue: $3,524,535.90
Average order value: $10.85


In [6]:
# Set analysis date (day after the last transaction date)
analysis_date = df_clean['InvoiceDate'].max() + timedelta(days=1)
print(f"Analysis Date: {analysis_date}")

# Calculate RFM metrics
rfm_data = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (analysis_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',  # Frequency
    'TotalAmount': 'sum'     # Monetary
}).reset_index()

# Rename columns for clarity
rfm_data.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Add additional customer metrics
customer_metrics = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': ['min', 'max'],
    'Quantity': 'sum',
    'StockCode': 'nunique',
    'Country': 'first'
}).reset_index()

customer_metrics.columns = ['CustomerID', 'FirstPurchase', 'LastPurchase', 
                           'TotalQuantity', 'UniqueProducts', 'Country']

# Merge RFM with additional metrics
rfm_complete = rfm_data.merge(customer_metrics, on='CustomerID')

# Calculate customer lifetime (days)
rfm_complete['CustomerLifetime'] = (rfm_complete['LastPurchase'] - 
                                   rfm_complete['FirstPurchase']).dt.days + 1

# Calculate average order value and purchase frequency
rfm_complete['AvgOrderValue'] = rfm_complete['Monetary'] / rfm_complete['Frequency']
rfm_complete['AvgDaysBetweenPurchases'] = rfm_complete['CustomerLifetime'] / rfm_complete['Frequency']

print(f"\nRFM Dataset Shape: {rfm_complete.shape}")
print(f"\nRFM Statistics:")
print(rfm_complete[['Recency', 'Frequency', 'Monetary']].describe())

# Save cleaned data
df_clean.to_csv('data/online_retail_cleaned.csv', index=False)
rfm_complete.to_csv('data/rfm_customer_data.csv', index=False)
print("\nCleaned datasets saved!")

Analysis Date: 2011-12-10 12:50:00

RFM Dataset Shape: (4146, 12)

RFM Statistics:
           Recency    Frequency      Monetary
count  4146.000000  4146.000000   4146.000000
mean     93.181380     3.918234    850.105138
std     100.015984     6.813902   1767.635549
min       1.000000     1.000000      1.700000
25%      18.000000     1.000000    173.970000
50%      51.000000     2.000000    405.625000
75%     145.000000     4.000000    964.527500
max     374.000000   193.000000  70409.910000

Cleaned datasets saved!


In [7]:
# Create RFM scores using quintiles (1-5 scale)
rfm_scores = rfm_complete.copy()

# Calculate quintile scores (5 = best, 1 = worst)
rfm_scores['R_Score'] = pd.qcut(rfm_scores['Recency'].rank(method='first'), 5, labels=[5,4,3,2,1])
rfm_scores['F_Score'] = pd.qcut(rfm_scores['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm_scores['M_Score'] = pd.qcut(rfm_scores['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5])

# Convert scores to integers
rfm_scores['R_Score'] = rfm_scores['R_Score'].astype(int)
rfm_scores['F_Score'] = rfm_scores['F_Score'].astype(int) 
rfm_scores['M_Score'] = rfm_scores['M_Score'].astype(int)

# Create combined RFM score
rfm_scores['RFM_Score'] = rfm_scores['R_Score'].astype(str) + \
                         rfm_scores['F_Score'].astype(str) + \
                         rfm_scores['M_Score'].astype(str)

# Calculate overall customer value score
rfm_scores['RFM_Value'] = rfm_scores['R_Score'] + rfm_scores['F_Score'] + rfm_scores['M_Score']

print("RFM Scoring Complete!")
print(f"RFM Score Distribution:")
print(rfm_scores['RFM_Value'].describe())

RFM Scoring Complete!
RFM Score Distribution:
count    4146.000000
mean        8.999518
std         3.579664
min         3.000000
25%         6.000000
50%         9.000000
75%        12.000000
max        15.000000
Name: RFM_Value, dtype: float64


In [8]:
# Define customer segments based on RFM scores
def segment_customers(row):
    if row['RFM_Value'] >= 13:
        return 'Champions'
    elif row['RFM_Value'] >= 10:
        if row['R_Score'] >= 4:
            return 'Loyal Customers'
        else:
            return 'At Risk'
    elif row['RFM_Value'] >= 7:
        if row['R_Score'] >= 3:
            return 'Potential Loyalists'
        else:
            return 'Cannot Lose Them'
    elif row['RFM_Value'] >= 5:
        if row['F_Score'] <= 2:
            return 'New Customers'
        else:
            return 'Promising'
    else:
        if row['R_Score'] >= 3:
            return 'Need Attention'
        else:
            return 'Lost Customers'

# Apply segmentation
rfm_scores['Customer_Segment'] = rfm_scores.apply(segment_customers, axis=1)

# Analyze segments
segment_analysis = rfm_scores.groupby('Customer_Segment').agg({
    'CustomerID': 'count',
    'Recency': 'mean',
    'Frequency': 'mean', 
    'Monetary': 'mean',
    'RFM_Value': 'mean'
}).round(2)

segment_analysis.columns = ['Customer_Count', 'Avg_Recency', 'Avg_Frequency', 
                           'Avg_Monetary', 'Avg_RFM_Score']

# Calculate segment percentages and revenue contribution
segment_analysis['Percentage'] = (segment_analysis['Customer_Count'] / len(rfm_scores) * 100).round(2)
segment_analysis['Total_Revenue'] = rfm_scores.groupby('Customer_Segment')['Monetary'].sum().round(2)
segment_analysis['Revenue_Percentage'] = (segment_analysis['Total_Revenue'] / rfm_scores['Monetary'].sum() * 100).round(2)

print("Customer Segmentation Analysis:")
print(segment_analysis.sort_values('Total_Revenue', ascending=False))

# Top customer segments by value
print(f"\nTop 3 Segments by Revenue:")
top_segments = segment_analysis.sort_values('Total_Revenue', ascending=False).head(3)
for segment in top_segments.index:
    revenue_pct = top_segments.loc[segment, 'Revenue_Percentage']
    customer_pct = top_segments.loc[segment, 'Percentage']
    print(f"{segment}: {revenue_pct}% of revenue from {customer_pct}% of customers")

Customer Segmentation Analysis:
                     Customer_Count  Avg_Recency  Avg_Frequency  Avg_Monetary  \
Customer_Segment                                                                
Champions                       895        14.76          10.37       2410.91   
At Risk                         437        75.13           4.47       1075.21   
Loyal Customers                 501        17.12           3.11        582.22   
Cannot Lose Them                433       147.88           2.42        517.01   
Potential Loyalists             611        35.86           1.56        285.10   
New Customers                   650       146.15           1.06        211.82   
Lost Customers                  498       253.82           1.01        106.79   
Promising                       121       201.39           2.13        134.55   

                     Avg_RFM_Score  Percentage  Total_Revenue  \
Customer_Segment                                                
Champions                  

In [9]:
# Prepare data for clustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Select features for clustering
clustering_features = ['Recency', 'Frequency', 'Monetary', 'AvgOrderValue', 'UniqueProducts']
X = rfm_scores[clustering_features].copy()

# Handle any potential missing values
X = X.fillna(X.median())

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Find optimal number of clusters using elbow method
inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# Apply K-Means with optimal clusters (let's use 5)
kmeans_final = KMeans(n_clusters=5, random_state=42, n_init=10)
rfm_scores['ML_Cluster'] = kmeans_final.fit_predict(X_scaled)

# Analyze ML clusters
cluster_analysis = rfm_scores.groupby('ML_Cluster').agg({
    'CustomerID': 'count',
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'AvgOrderValue': 'mean',
    'UniqueProducts': 'mean'
}).round(2)

print(f"\nMachine Learning Cluster Analysis:")
print(cluster_analysis)

# Save segmented data
rfm_scores.to_csv('output/rfm_customer_segments.csv', index=False)
segment_analysis.to_csv('output/customer_segment_analysis.csv')
print(f"\nSegmentation analysis saved!")


Machine Learning Cluster Analysis:
            CustomerID  Recency  Frequency  Monetary  AvgOrderValue  \
ML_Cluster                                                            
0                  449    64.15       2.98   1526.03         563.91   
1                 2424    44.34       3.45    574.53         172.00   
2                  990   248.43       1.48    224.88         158.09   
3                    3     1.33     168.00  40040.93         240.35   
4                  280    14.62      16.31   3942.59         279.14   

            UniqueProducts  
ML_Cluster                  
0                    93.77  
1                    41.14  
2                    19.22  
3                  1436.33  
4                   216.01  

Segmentation analysis saved!


In [10]:
# Define churn based on recency (customers who haven't purchased in 90+ days)
rfm_churn = rfm_scores.copy()

# Create churn label (1 = churned, 0 = active)
churn_threshold = 90  # days
rfm_churn['Churned'] = (rfm_churn['Recency'] > churn_threshold).astype(int)

# Create additional features for churn prediction
rfm_churn['DaysActive'] = rfm_churn['CustomerLifetime']
rfm_churn['PurchaseFrequency'] = rfm_churn['Frequency'] / (rfm_churn['CustomerLifetime'] / 30)  # purchases per month
rfm_churn['MonthsSinceFirstPurchase'] = rfm_churn['CustomerLifetime'] / 30
rfm_churn['IsOneTimeBuyer'] = (rfm_churn['Frequency'] == 1).astype(int)

# Calculate churn rate
churn_rate = rfm_churn['Churned'].mean()
print(f"Overall Churn Rate: {churn_rate:.2%}")

# Churn by segment analysis
churn_by_segment = rfm_churn.groupby('Customer_Segment').agg({
    'Churned': ['count', 'sum', 'mean'],
    'Monetary': 'sum'
}).round(3)
churn_by_segment.columns = ['Total_Customers', 'Churned_Customers', 'Churn_Rate', 'Revenue_At_Risk']
churn_by_segment['Revenue_At_Risk'] = churn_by_segment['Revenue_At_Risk'] * churn_by_segment['Churn_Rate']

print(f"\nChurn Analysis by Customer Segment:")
print(churn_by_segment.sort_values('Churn_Rate', ascending=False))

Overall Churn Rate: 33.53%

Churn Analysis by Customer Segment:
                     Total_Customers  Churned_Customers  Churn_Rate  \
Customer_Segment                                                      
Lost Customers                   498                480       0.964   
Promising                        121                112       0.926   
Cannot Lose Them                 433                329       0.760   
New Customers                    650                365       0.562   
At Risk                          437                104       0.238   
Champions                        895                  0       0.000   
Loyal Customers                  501                  0       0.000   
Potential Loyalists              611                  0       0.000   

                     Revenue_At_Risk  
Customer_Segment                      
Lost Customers          51268.450560  
Promising               15076.280080  
Cannot Lose Them       170139.149520  
New Customers           77379.

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder

# Prepare features for ML model
feature_columns = ['Recency', 'Frequency', 'Monetary', 'AvgOrderValue', 
                  'UniqueProducts', 'CustomerLifetime', 'PurchaseFrequency', 
                  'MonthsSinceFirstPurchase', 'IsOneTimeBuyer', 'RFM_Value']

X = rfm_churn[feature_columns].copy()
y = rfm_churn['Churned'].copy()

# Handle missing values
X = X.fillna(X.median())

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

# Model evaluation
print(f"\nChurn Prediction Model Performance:")
print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_proba):.3f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nTop 5 Features for Churn Prediction:")
print(feature_importance.head())

# Add churn probability to dataset
rfm_churn['Churn_Probability'] = rf_model.predict_proba(X)[:, 1]
rfm_churn['Churn_Risk_Level'] = pd.cut(rfm_churn['Churn_Probability'], 
                                      bins=[0, 0.3, 0.7, 1.0], 
                                      labels=['Low', 'Medium', 'High'])


Churn Prediction Model Performance:
ROC AUC Score: 1.000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       552
           1       1.00      1.00      1.00       278

    accuracy                           1.00       830
   macro avg       1.00      1.00      1.00       830
weighted avg       1.00      1.00      1.00       830


Top 5 Features for Churn Prediction:
                    Feature  Importance
0                   Recency    0.728410
9                 RFM_Value    0.151187
5          CustomerLifetime    0.036547
7  MonthsSinceFirstPurchase    0.029297
2                  Monetary    0.014761


In [12]:
# Calculate Customer Lifetime Value (CLV)
# CLV = Average Order Value × Purchase Frequency × Customer Lifespan

# Average customer lifespan in months (using historical data)
avg_customer_lifespan_months = rfm_churn['MonthsSinceFirstPurchase'].mean()

# Calculate CLV for each customer
rfm_churn['Monthly_Purchase_Rate'] = rfm_churn['Frequency'] / rfm_churn['MonthsSinceFirstPurchase']
rfm_churn['Monthly_Revenue'] = rfm_churn['AvgOrderValue'] * rfm_churn['Monthly_Purchase_Rate']
rfm_churn['CLV'] = rfm_churn['Monthly_Revenue'] * avg_customer_lifespan_months

# CLV by segment
clv_analysis = rfm_churn.groupby('Customer_Segment').agg({
    'CLV': ['mean', 'median', 'sum'],
    'CustomerID': 'count',
    'Churn_Probability': 'mean'
}).round(2)
clv_analysis.columns = ['Avg_CLV', 'Median_CLV', 'Total_CLV', 'Customer_Count', 'Avg_Churn_Risk']

print(f"\nCustomer Lifetime Value Analysis:")
print(clv_analysis.sort_values('Total_CLV', ascending=False))

# High-value at-risk customers
high_risk_valuable = rfm_churn[
    (rfm_churn['CLV'] > rfm_churn['CLV'].quantile(0.8)) & 
    (rfm_churn['Churn_Risk_Level'] == 'High')
][['CustomerID', 'Customer_Segment', 'CLV', 'Churn_Probability', 'Monetary']].sort_values('CLV', ascending=False)

print(f"\nHigh-Value At-Risk Customers (Top 10):")
print(high_risk_valuable.head(10))

# Save comprehensive analysis
rfm_churn.to_csv('output/comprehensive_customer_analysis.csv', index=False)
churn_by_segment.to_csv('output/churn_analysis_by_segment.csv')
clv_analysis.to_csv('output/customer_lifetime_value_analysis.csv')
feature_importance.to_csv('output/churn_prediction_feature_importance.csv', index=False)
high_risk_valuable.to_csv('output/high_value_at_risk_customers.csv', index=False)

print(f"\nComprehensive customer analysis saved to output folder!")


Customer Lifetime Value Analysis:
                      Avg_CLV  Median_CLV    Total_CLV  Customer_Count  \
Customer_Segment                                                         
New Customers        25538.30    21737.45  16599895.52             650   
Potential Loyalists  20787.86    12963.58  12701380.24             611   
Lost Customers       13671.89    11836.17   6808600.79             498   
Cannot Lose Them     12939.20      569.01   5602675.65             433   
Loyal Customers       3272.75      442.13   1639645.55             501   
Champions             1642.61      892.71   1470140.31             895   
At Risk               2905.08      633.89   1269521.88             437   
Promising             2531.91      262.77    306361.45             121   

                     Avg_Churn_Risk  
Customer_Segment                     
New Customers                  0.56  
Potential Loyalists            0.00  
Lost Customers                 0.96  
Cannot Lose Them               0.7

In [13]:
# 1. Main customer dashboard dataset
powerbi_main = rfm_churn[['CustomerID', 'Country', 'Customer_Segment', 'ML_Cluster',
                         'Recency', 'Frequency', 'Monetary', 'RFM_Value',
                         'AvgOrderValue', 'UniqueProducts', 'CustomerLifetime',
                         'CLV', 'Churn_Probability', 'Churn_Risk_Level', 'Churned',
                         'FirstPurchase', 'LastPurchase']].copy()

# Add categorical variables for better Power BI visualization
powerbi_main['RFM_Category'] = pd.cut(powerbi_main['RFM_Value'], 
                                     bins=[0, 5, 8, 11, 15], 
                                     labels=['Low Value', 'Medium Value', 'High Value', 'Premium'])

powerbi_main['CLV_Category'] = pd.qcut(powerbi_main['CLV'], 
                                      q=4, 
                                      labels=['Low CLV', 'Medium CLV', 'High CLV', 'Premium CLV'])

powerbi_main['Recency_Category'] = pd.cut(powerbi_main['Recency'], 
                                         bins=[0, 30, 90, 180, 400], 
                                         labels=['Active', 'Recent', 'Dormant', 'Lost'])

# 2. Segment performance summary for KPI cards
segment_kpis = rfm_churn.groupby('Customer_Segment').agg({
    'CustomerID': 'count',
    'Monetary': ['sum', 'mean'],
    'Frequency': 'mean',
    'Recency': 'mean',
    'CLV': ['sum', 'mean'],
    'Churn_Probability': 'mean',
    'AvgOrderValue': 'mean'
}).round(2)

segment_kpis.columns = ['Customer_Count', 'Total_Revenue', 'Avg_Revenue_Per_Customer',
                       'Avg_Frequency', 'Avg_Recency', 'Total_CLV', 'Avg_CLV', 
                       'Avg_Churn_Risk', 'Avg_Order_Value']

segment_kpis = segment_kpis.reset_index()

# Add percentage calculations
segment_kpis['Customer_Percentage'] = (segment_kpis['Customer_Count'] / segment_kpis['Customer_Count'].sum() * 100).round(2)
segment_kpis['Revenue_Percentage'] = (segment_kpis['Total_Revenue'] / segment_kpis['Total_Revenue'].sum() * 100).round(2)
segment_kpis['CLV_Percentage'] = (segment_kpis['Total_CLV'] / segment_kpis['Total_CLV'].sum() * 100).round(2)

# 3. Time-based analysis (monthly cohorts)
df_clean_time = df_clean.copy()
df_clean_time['YearMonth'] = df_clean_time['InvoiceDate'].dt.to_period('M')

monthly_cohorts = df_clean_time.groupby(['CustomerID', 'YearMonth']).agg({
    'TotalAmount': 'sum',
    'InvoiceNo': 'nunique',
    'Quantity': 'sum'
}).reset_index()

monthly_cohorts.columns = ['CustomerID', 'YearMonth', 'Monthly_Revenue', 'Monthly_Orders', 'Monthly_Quantity']
monthly_cohorts['YearMonth'] = monthly_cohorts['YearMonth'].astype(str)

# Merge with customer segments
monthly_cohorts = monthly_cohorts.merge(
    powerbi_main[['CustomerID', 'Customer_Segment', 'CLV_Category', 'Country']], 
    on='CustomerID'
)

# 4. Country performance analysis
country_analysis = rfm_churn.groupby('Country').agg({
    'CustomerID': 'count',
    'Monetary': 'sum',
    'CLV': 'sum',
    'Churn_Probability': 'mean',
    'RFM_Value': 'mean'
}).round(2)

country_analysis.columns = ['Total_Customers', 'Total_Revenue', 'Total_CLV', 'Avg_Churn_Risk', 'Avg_RFM_Score']
country_analysis = country_analysis.reset_index()

print("Power BI datasets created:")
print(f"Main customer dataset: {powerbi_main.shape}")
print(f"Segment KPIs: {segment_kpis.shape}")
print(f"Monthly cohorts: {monthly_cohorts.shape}")
print(f"Country analysis: {country_analysis.shape}")

Power BI datasets created:
Main customer dataset: (4146, 20)
Segment KPIs: (8, 13)
Monthly cohorts: (12100, 8)
Country analysis: (37, 6)


In [14]:
# Export to CSV files for Power BI
powerbi_main.to_csv('output/powerbi_customer_analysis.csv', index=False)
segment_kpis.to_csv('output/powerbi_segment_kpis.csv', index=False)
monthly_cohorts.to_csv('output/powerbi_monthly_cohorts.csv', index=False)
country_analysis.to_csv('output/powerbi_country_analysis.csv', index=False)

# Create high-priority customer lists for targeted analysis
champions_list = rfm_churn[rfm_churn['Customer_Segment'] == 'Champions'][
    ['CustomerID', 'Country', 'Monetary', 'Frequency', 'CLV', 'AvgOrderValue']
].sort_values('CLV', ascending=False)

at_risk_high_value = rfm_churn[
    (rfm_churn['Customer_Segment'].isin(['At Risk', 'Cannot Lose Them'])) &
    (rfm_churn['CLV'] > rfm_churn['CLV'].median())
][['CustomerID', 'Customer_Segment', 'Country', 'Monetary', 'CLV', 'Churn_Probability']
 ].sort_values('CLV', ascending=False)

champions_list.to_csv('output/powerbi_champions_customers.csv', index=False)
at_risk_high_value.to_csv('output/powerbi_at_risk_high_value.csv', index=False)

# Create Power BI data dictionary
data_dictionary = """
# Power BI Data Dictionary - Customer Segmentation Analysis

## powerbi_customer_analysis.csv (Main Dataset)
- CustomerID: Unique customer identifier
- Country: Customer country
- Customer_Segment: RFM-based segment (Champions, At Risk, etc.)
- ML_Cluster: Machine learning cluster assignment (0-4)
- Recency: Days since last purchase
- Frequency: Total number of orders
- Monetary: Total customer revenue
- RFM_Value: Combined RFM score (3-15)
- AvgOrderValue: Average order amount
- UniqueProducts: Number of different products purchased
- CLV: Customer Lifetime Value prediction
- Churn_Probability: ML-predicted churn probability (0-1)
- Churn_Risk_Level: Low/Medium/High risk categories
- RFM_Category: Low/Medium/High/Premium value tiers
- CLV_Category: CLV quartile groupings
- Recency_Category: Active/Recent/Dormant/Lost status

## powerbi_segment_kpis.csv (KPI Dashboard)
- Customer_Segment: Segment name
- Customer_Count: Number of customers in segment
- Total_Revenue: Segment revenue contribution
- Avg_Revenue_Per_Customer: Average customer value
- Customer_Percentage: % of total customer base
- Revenue_Percentage: % of total revenue
- CLV_Percentage: % of total predicted CLV
- Avg_Churn_Risk: Average churn probability in segment

## powerbi_monthly_cohorts.csv (Time Analysis)
- CustomerID: Customer identifier
- YearMonth: Transaction period (YYYY-MM format)
- Monthly_Revenue: Customer revenue in that month
- Monthly_Orders: Number of orders in month
- Customer_Segment: Customer's segment classification
- CLV_Category: Customer's CLV tier

## powerbi_country_analysis.csv (Geographic Analysis)
- Country: Country name
- Total_Customers: Number of customers
- Total_Revenue: Country revenue contribution
- Total_CLV: Predicted CLV for country
- Avg_Churn_Risk: Average churn risk in country
- Avg_RFM_Score: Average RFM performance
"""

with open('output/powerbi_data_dictionary.txt', 'w') as f:
    f.write(data_dictionary)

print("\nAll Power BI datasets exported successfully:")
print("- powerbi_customer_analysis.csv (4,146 customers)")
print("- powerbi_segment_kpis.csv (8 segments)")
print("- powerbi_monthly_cohorts.csv (monthly analysis)")
print("- powerbi_country_analysis.csv (geographic analysis)")
print("- powerbi_champions_customers.csv (top customers)")
print("- powerbi_at_risk_high_value.csv (retention focus)")
print("- powerbi_data_dictionary.txt")
print("\nReady for Power BI dashboard creation!")

# Display key insights for dashboard design
print(f"\nKey Insights for Dashboard:")
print(f"• Champions segment: {segment_kpis[segment_kpis['Customer_Segment']=='Champions']['Revenue_Percentage'].iloc[0]}% of revenue")
print(f"• At-risk revenue: ${rfm_churn[rfm_churn['Churn_Risk_Level']=='High']['Monetary'].sum():,.0f}")
print(f"• Average CLV: ${rfm_churn['CLV'].mean():,.0f}")
print(f"• Customers needing attention: {len(at_risk_high_value)} high-value at-risk customers")


All Power BI datasets exported successfully:
- powerbi_customer_analysis.csv (4,146 customers)
- powerbi_segment_kpis.csv (8 segments)
- powerbi_monthly_cohorts.csv (monthly analysis)
- powerbi_country_analysis.csv (geographic analysis)
- powerbi_champions_customers.csv (top customers)
- powerbi_at_risk_high_value.csv (retention focus)
- powerbi_data_dictionary.txt

Ready for Power BI dashboard creation!

Key Insights for Dashboard:
• Champions segment: 61.22% of revenue
• At-risk revenue: $457,758
• Average CLV: $11,191
• Customers needing attention: 203 high-value at-risk customers
